In [3]:
#imports
import config
import requests
import json
import csv
import pprint
import pandas as pd
import time


# pretty printer for nicer formating when needed
pp = pprint.PrettyPrinter(indent = 1)


In [18]:
# Here we define our query as a multi-line string
query = '''
query ($id: Int, $page: Int, $perPage: Int, $search: String) {
    Page (page: $page, perPage: $perPage) {
        pageInfo {
            total
            currentPage
            lastPage
            hasNextPage
            perPage
        }
        media (id: $id, search: $search) {
            id
            title {
                romaji
                english
                native
            }
            synonyms
            startDate {
                year
                month
                day
            }
            description
            countryOfOrigin
            averageScore
            meanScore
            genres
            popularity
            favourites
            source
            isAdult
        }
    }
}
'''
# 1571 pages with 50 per page
variables = {
    'page': 1,
    'perPage' : 50
}

url = 'https://graphql.anilist.co'

In [19]:
response = pd.DataFrame((requests.post(url, json={'query': query, 'variables': variables}).json())['data']['Page']['media'])

In [21]:
anime = pd.DataFrame()

for i in range(1, 1572):
    variables['page'] = i
    try:
        temp = pd.DataFrame((requests.post(url, json = {'query': query, 'variables': variables}).json())['data']['Page']['media'])
        anime = pd.concat([anime, temp], ignore_index = True)
    except:
        time.sleep(120)
        i -= 1

In [22]:
anime.to_excel("anilist_raw.xlsx", index = False)
anime.to_csv("anilist_raw.csv", index = False)